# Exercise 5: Lasso Regression on the Franke function  with resampling (Score 10 points))


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os
from common import *
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

os.chdir("../")
print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

%matplotlib inline 

Root directory: /home/arefrode/uio/fys-stk4155/projects/FYS-STK4155-Prj1_report


# 1.5
Start off by defining the data

In [2]:
np.random.seed(SEED_VALUE)
maxdegree = 12
degree = 5 # Chosen as the optimal degree from the bias-variance tentative
n = 400
x = np.sort(np.random.uniform(0, 1, n))
y = np.sort(np.random.uniform(0, 1, n))
z = FrankeFunction(x, y) + 0.2*np.random.normal(0, size = n)

4155
